In [1]:
!pip install tensorflow

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Step 1: Load the data
data = pd.read_csv("ecommerce_user_behaviorX.csv")

In [17]:
# Step 2: Preprocessing
# Handling missing values, if any
data.fillna(method='ffill', inplace=True)

In [18]:
# Encode categorical variables
label_encoders = {}
for column in ['User ID', 'Device Type', 'Location']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

In [19]:

# Target variable encoding
target_encoder = LabelEncoder()
data['Authentication Method'] = target_encoder.fit_transform(data['Authentication Method'])

# Splitting the data into features and target
X = data.drop('Authentication Method', axis=1)
y = to_categorical(data['Authentication Method'])

In [20]:
# Standardizing the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# Step 3: Building the model
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')
])

In [22]:
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Step 4: Training the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
600/600 [==============================] - 2s 2ms/step - loss: 0.7090 - accuracy: 0.6754 - val_loss: 0.4626 - val_accuracy: 0.7283
Epoch 2/50
600/600 [==============================] - 1s 2ms/step - loss: 0.4168 - accuracy: 0.7438 - val_loss: 0.4073 - val_accuracy: 0.7365
Epoch 3/50
600/600 [==============================] - 1s 2ms/step - loss: 0.3883 - accuracy: 0.7498 - val_loss: 0.3925 - val_accuracy: 0.7352
Epoch 4/50
600/600 [==============================] - 1s 2ms/step - loss: 0.3789 - accuracy: 0.7534 - val_loss: 0.3868 - val_accuracy: 0.7358
Epoch 5/50
600/600 [==============================] - 2s 4ms/step - loss: 0.3736 - accuracy: 0.7585 - val_loss: 0.3851 - val_accuracy: 0.7275
Epoch 6/50
600/600 [==============================] - 1s 2ms/step - loss: 0.3695 - accuracy: 0.7610 - val_loss: 0.3938 - val_accuracy: 0.7433
Epoch 7/50
600/600 [==============================] - 1s 2ms/step - loss: 0.3689 - accuracy: 0.7598 - val_loss: 0.3816 - val_accuracy: 0.7342
Epoch 

In [24]:
# Step 5: Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

188/188 [==============================] - 0s 1ms/step - loss: 0.3934 - accuracy: 0.7420
Test Accuracy: 74.20%


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Predicting on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculating the metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


188/188 [==============================] - 0s 1ms/step
Accuracy: 74.20%
Precision: 0.78
Recall: 0.77
F1 Score: 0.77


In [29]:
def safe_encode(column, encoder, default='Unknown'):
    try:
        return encoder.transform(column)
    except ValueError:
        # If the category is unknown, assign a default label (e.g., the last label + 1)
        return [len(encoder.classes_)] * len(column)

# Modify the existing make_prediction function to handle unknown categories safely
def make_prediction(input_data):
    df = pd.DataFrame([input_data], columns=['User ID', 'Session Duration', 'Pages Visited', 'Items Viewed',
                                             'Login Frequency', 'Cart Additions', 'Successful Transactions',
                                             'Failed Transactions', 'Device Type', 'Age', 'Location'])

    # Apply encoding safely
    df['Location'] = safe_encode(df['Location'], label_encoders['Location'])
    df['Device Type'] = safe_encode(df['Device Type'], label_encoders['Device Type'])
    df['User ID'] = safe_encode(df['User ID'], label_encoders['User ID'])

    # Standard scaling
    df = scaler.transform(df)

    # Make prediction
    prediction = model.predict(df)
    predicted_class = np.argmax(prediction)
    predicted_auth_method = target_encoder.inverse_transform([predicted_class])[0]
    return predicted_auth_method

# Example input data
input_data = {
    'User ID': 50, 'Session Duration': 10, 'Pages Visited': 2, 'Items Viewed': 1,
    'Login Frequency': 1, 'Cart Additions': 2, 'Successful Transactions': 1,
    'Failed Transactions': 0, 'Device Type': 'Mobile', 'Age': 30, 'Location': 'USA'
}

predicted_auth_method = make_prediction(input_data)
print(f"Predicted Authentication Method: {predicted_auth_method}")

1/1 [==============================] - 0s 64ms/step
Predicted Authentication Method: OTP


In [30]:
# Optionally save the model
model.save('authentication_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
